# Hosting a model as a REST Endpoint 

This example gives an easy introduction into how to host your trained models and make them accesible as REST Endpoints.

## Set Up Training and Test Set

In [3]:
from sklearn.preprocessing import normalize 
from amorf.datasets import RiverFlow1 
from sklearn.model_selection import train_test_split

X, y = RiverFlow1().get_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)



## Initialize and Train Estimator

In [4]:
from amorf.problemTransformation import SingleTargetMethod
from amorf.metrics import average_relative_root_mean_squared_error

regressor = SingleTargetMethod('gradientboost')
regressor.fit(X_train, y_train) 


/home/davidhildner/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
0.22651400661633916


## Perform Prediction and Calculate Error 


In [ ]:
result = regressor.predict(X_test)  
print(average_relative_root_mean_squared_error(result, y_test))

## Setting Up Endpoint using Flask

In [16]:
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin
import numpy as np

app = Flask(__name__)
CORS(app)

@app.route("/predict", methods=['POST'])
def predict():
    payload = request.json['data']
    value = np.array(payload).reshape(1,-1)   
    
    prediction = regressor.predict(value)
    
    ret = '{"prediction":' + str(prediction) + '}'
    
    return ret

# running REST API on 127.0.0.1:5000
if __name__ == "__main__":
    app.run(debug=False, host='127.0.0.1', port=5000)

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
/home/davidhildner/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested below threads, setting n_jobs=1
  **self._backend_args)
127.0.0.1 - - [05/Jan/2020 17:07:51] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2020 17:08:03] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2020 17:08:14] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2020 17:08:16] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2020 17:08:23] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2020 17:08:25] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2020 17:08:26] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2020 17:08:27] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2020 17:08:27] "PO

## Use Curl to get Prediction

If you send a Post request using Curl (or any other request library or tool) to the endpoint containing a new sample it will return prediction. 
 
```bash 
curl -i -X POST \
   -H "Content-Type:application/json" \
   -d \
'{
  "data": [265,  3.26,   256,   183,  62.4,  8.11,   157,    57,   264,  3.27,   257,   185,  61.4,  8.42,   156,  57.6,   268,  3.33,   257,   185,  61.2,  8.21,   156,  55.9,   269,  3.35,   257,   186,  60.9,  8.04,   157,  54.9,   272,  3.36,   259,   186,  61.2,  8.07,   158,  54.9,   274,  3.34,   261,   187,  61.3,  8.18,   159,  55.5,   283,  3.36,   264,   190,  63.3,  8.42,   162,  55.8,   287,  3.39,   273,   193,    64,   8.7,   164,  57.9]
}' \
 'http://localhost:5000/predict'```